# AIR - Exercise in Google Colab

## Colab Preparation

Open via google drive -> right click: open with Colab

**Get a GPU**

Toolbar -> Runtime -> Change Runtime Type -> GPU

**Mount Google Drive**

* Download data and clone your github repo to your Google Drive folder
* Use Google Drive as connection between Github and Colab (Could also use direct github access, but re-submitting credentials might be annoying)
* Commit to Github locally from the synced drive

**Keep Alive**

When training google colab tends to kick you out, This might help: https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0

**Get Started**

Run the following script to mount google drive and install needed python packages. Pytorch comes pre-installed.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/AIR/src
import sys
sys.path.append('/content/drive/MyDrive/AIR/src')

Mounted at /content/drive
/content/drive/MyDrive/AIR/src


In [2]:
from extractive_qa import ExtractiveQA

## Load the model and the extractor

In [ ]:
model_name1 = "bert-large-uncased-whole-word-masking-finetuned-squad"

extractor = ExtractiveQA(model_name1)

## Load the data

In [4]:
top1_results_path = '../results/top1_results_tk.csv'
top2_results_path = '../results/top2_results_tk.csv'
fira_answers_path = '../data/Part-3/msmarco-fira-21.qrels.qa-answers.tsv'
fira_tuples_path = '../data/Part-3/msmarco-fira-21.qrels.qa-tuples.tsv'
output_path = '../data/Part-3/extractive_qa_results.csv'

In [5]:
top1_results = extractor.load_top1_results(top1_results_path)
top2_results = extractor.load_top1_results(top2_results_path)
fira_answers, fira_tuples = extractor.load_fira_gold_labels(fira_answers_path, fira_tuples_path)

In [17]:
fira_tuples.head(5)

,query_id,doc_id,relevance,query_text,document_text,answer
0,135386,100163,3,definition of imagination,imagination - the formation of a mental image ...,the formation of a mental image of something t...
1,290779,101026,3,how many oscars has clint eastwood won?pdrijgh...,Clint Eastwood -- five-time Oscar winner and e...,five
2,21741,1021598,3,are cold sores and fever blisters the same,"Cold sores, sometimes called fever blisters, a...","Cold sores, sometimes called fever blisters"
3,810210,1029662,3,what is the cause of blood in the stool,Having blood in the stool can be the result of...,"wide variety of conditions, such as hemorrhoid..."
4,1097448,103635,3,how many calories in slim fast shakes,"The chocolate-flavored shake contains 190, whi...",chocolate-flavored shake contains 190\tCafe Cl...


## Run the extractive QA

In [8]:
results_top1 = extractor.run_extractive_qa(top1_results, fira_tuples)

Processing row 0 of 2000
Processing row 100 of 2000
Processing row 200 of 2000
Processing row 300 of 2000
Processing row 400 of 2000
Processing row 500 of 2000
Processing row 600 of 2000
Processing row 700 of 2000
Processing row 800 of 2000
Processing row 900 of 2000
Processing row 1000 of 2000
Processing row 1100 of 2000
Processing row 1200 of 2000
Processing row 1300 of 2000
Processing row 1400 of 2000
Processing row 1500 of 2000
Processing row 1600 of 2000
Processing row 1700 of 2000
Processing row 1800 of 2000
Processing row 1900 of 2000
Missing query_ids: 8
Missing doc_ids: 951


In [9]:
results_top2 = extractor.run_extractive_qa2(top2_results, fira_tuples)

Processing row 0 of 2000
Processing row 100 of 2000
Processing row 200 of 2000
Processing row 300 of 2000
Processing row 400 of 2000
Processing row 500 of 2000
Processing row 600 of 2000
Processing row 700 of 2000
Processing row 800 of 2000
Processing row 900 of 2000
Processing row 1000 of 2000
Processing row 1100 of 2000
Processing row 1200 of 2000
Processing row 1300 of 2000
Processing row 1400 of 2000
Processing row 1500 of 2000
Processing row 1600 of 2000
Processing row 1700 of 2000
Processing row 1800 of 2000
Processing row 1900 of 2000
Missing query_ids: 16
Missing doc_ids: 3046


In [13]:
extractor.save_results(results_top1, '../data/Part-3/extractive_qa_results_top1.csv')
extractor.save_results(results_top2, '../data/Part-3/extractive_qa_results_top2.csv')

## Demonstrate the answers to 5 queries from documents

In [11]:
# Show some examples of answers
# Take 5 random top1_results
for i in range(5):
    query_id = results_top2['query_id'][i]
    doc_id = results_top2['doc_id'][i]
    query_text = fira_tuples[fira_tuples["query_id"] == query_id]['query_text'].values[0]
    document_text = fira_tuples[fira_tuples["doc_id"] == doc_id]['document_text'].values[0]
    print(f"Query: {query_text}")
    print(f"Document: {document_text}")
    print(f"Answer: {results_top2['answer'][i]}")
    print()

Query: What Does Noel Mean in the Bible
Document: Period extending from Dec. 24 to Jan. 6.What the period was used for I have been unable to find out. Another answer: The word noel has Indo-European roots, and means birth or to be born.. It does not appear in the Bible. Another Answer Noel noël is Simply french For. christmas
Answer: It does not appear in the Bible

Query: _____ is the ability of cardiac pacemaker cells to spontaneously initiate an electrical impulse without being stimulated from another source, such as a nerve.
Document: A skeletal muscle contracts only after it is stimulated by a nerve. 2. Heart has pacemaker cells that can generate an electrical impulse without being stimulated by a nerve. a. ability of cardiac pacemaker cells to create an electrical impulse without being stimulated from another source is called automaticity.
Answer: automaticity

Query: access parallels cost
Document: Parallels Access makes using desktop programs touch-friendly (pictures) It's impo

## Evaluation

In [15]:
metrics = extractor.evaluate_results(results_top1, fira_answers)
print("Evaluation Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value}")

Gold answers: ['Testosterone is also essential for sperm maturation and sex drive.']
Predicted answer: Hormone
Jaccard score: 0.0
Exact match scores: [0]
F1 scores: [0]

Gold answers: [' If a president becomes unable to serve, whether through death, severe illness, or impeachment, the vice president would assume the position of president.']
Predicted answer: the vice president
Jaccard score: 0.1111111111111111
Exact match scores: [0]
F1 scores: [0.18181818181818182]

Gold answers: ['$482']
Predicted answer: average new-car loan payment hit a record $482 the fourth quarter
Jaccard score: 0.1111111111111111
Exact match scores: [0]
F1 scores: [0.19999999999999998]

Gold answers: ['Replacement costs between $378 and $860 on average']
Predicted answer: A Cadillac De Ville Alternator Replacement
Jaccard score: 0.08333333333333333
Exact match scores: [0]
F1 scores: [0.15384615384615385]

Gold answers: ['A partial hospitalization program (PHP) is a level of care that provides clients with acce

In [16]:
metrics = extractor.evaluate_results(results_top2, fira_answers)
print("Evaluation Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value}")

Gold answers: ['…what we, as a society, do collectively to assure the conditions in which people can be healthy']
Predicted answer: Concepts
Jaccard score: 0.0
Exact match scores: [0]
F1 scores: [0]

Gold answers: ['Testosterone is also essential for sperm maturation and sex drive.']
Predicted answer: Hormone
Jaccard score: 0.0
Exact match scores: [0]
F1 scores: [0]

Gold answers: [' If a president becomes unable to serve, whether through death, severe illness, or impeachment, the vice president would assume the position of president.']
Predicted answer: the vice president
Jaccard score: 0.1111111111111111
Exact match scores: [0]
F1 scores: [0.18181818181818182]

Gold answers: ['$482']
Predicted answer: average new-car loan payment hit a record $482 the fourth quarter
Jaccard score: 0.1111111111111111
Exact match scores: [0]
F1 scores: [0.19999999999999998]

Gold answers: ['an initiating document presented to a court clerk to be officially issued on behalf of the court or a the coverin